<a href="https://colab.research.google.com/github/hoiyurie/tibame-ml0602/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from urllib.request import urlretrieve
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/train.csv"
urlretrieve(url,"train.csv")
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/test.csv"
urlretrieve(url,"test.csv")


('test.csv', <http.client.HTTPMessage at 0x7fc3f9bc9f90>)

In [13]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding="utf-8")
test_df = pd.read_csv("test.csv",encoding="utf-8")

In [14]:
# 先把train data 和test data 兩個表格連結起來
data = pd.concat([train_df, test_df],ignore_index = True) # ingnore_index = True 重置合併後的index
data = data.drop(["PassengerId","Survived"], axis = 1)
data

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [15]:
# 查看空值 data.isna（）
# 查看空值總數 data.isna().sum()
# 對Series做篩選[帶入跟你的資料筆數一樣多True/False list]

na = data.isna().sum()
na[na>0].sort_values(ascending= False) # ascending = False 
na


Pclass         0
Name           0
Sex            0
Age          263
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin       1014
Embarked       2
dtype: int64